### Importing the merged .csv as a DataFrame

In [43]:
import pandas as pd
import numpy as np
from datetime import datetime

In [44]:
pd.set_option('display.max_columns', 50)

In [45]:
df = pd.read_csv('../data/merged_data.csv', index_col=0)

/home/linuxbrew/.linuxbrew/opt/python/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/linuxbrew/.linuxbrew/opt/python/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


### Checking columns names and dtypes

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1504150 entries, 0 to 1504149
Data columns (total 33 columns):
Accident_Index                                 1504150 non-null object
Location_Easting_OSGR                          1504049 non-null float64
Location_Northing_OSGR                         1504049 non-null float64
Longitude                                      1504049 non-null float64
Latitude                                       1504049 non-null float64
Police_Force                                   1504150 non-null int64
Accident_Severity                              1504150 non-null int64
Number_of_Vehicles                             1504150 non-null int64
Number_of_Casualties                           1504150 non-null int64
Date                                           1504150 non-null object
Day_of_Week                                    1504150 non-null int64
Time                                           1504033 non-null object
Local_Authority_(District)          

### Checking nan values

In [47]:
df.isna().sum()

Accident_Index                                       0
Location_Easting_OSGR                              101
Location_Northing_OSGR                             101
Longitude                                          101
Latitude                                           101
Police_Force                                         0
Accident_Severity                                    0
Number_of_Vehicles                                   0
Number_of_Casualties                                 0
Date                                                 0
Day_of_Week                                          0
Time                                               117
Local_Authority_(District)                           0
Local_Authority_(Highway)                            0
1st_Road_Class                                       0
1st_Road_Number                                      0
Road_Type                                            0
Speed_limit                                          0
Junction_D

### Columns to be dropped.
- Accident_Index: Not required since we can simply use the DataFrame index.
- Location_Easting_OSGR and Location_Northing_OSGR: Those are GB-specific coordinates. We have Lat and Lon and therefore we do not need them.
- Police_Force: No info.
- Local_Authority_(District) and Local_Authority_(Highway): Not useful for our analysis.
- 1st_Road_Class and 1st_Road_Number: Only related to junctions. We already have many attributes that are useful for our analysis and prediction.
- Junction_Detail and Junction_Control: almost no data.
- 2nd_Road_Class and 2nd_Road_Number: Same as above.
- Pedestrian_Crossing-Human_Control: Very specific information, not useful for our analysis.
- Special_Conditions_at_Site: We already have a lot of information on conditions.
- LSOA_of_Accident_Location: Again, we have Lat and Lon.

In [48]:
columns_to_drop = ['Accident_Index', 'Location_Easting_OSGR', 'Location_Northing_OSGR', 'Police_Force', 
                   'Local_Authority_(District)', 'Local_Authority_(Highway)', '1st_Road_Class',
                   '1st_Road_Number', 'Junction_Detail', 'Junction_Control', '2nd_Road_Class',
                   '2nd_Road_Number', 'Pedestrian_Crossing-Human_Control', 'Special_Conditions_at_Site',
                   'LSOA_of_Accident_Location']

In [49]:
df.drop(columns_to_drop, axis=1, inplace=True)

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1504150 entries, 0 to 1504149
Data columns (total 18 columns):
Longitude                                      1504049 non-null float64
Latitude                                       1504049 non-null float64
Accident_Severity                              1504150 non-null int64
Number_of_Vehicles                             1504150 non-null int64
Number_of_Casualties                           1504150 non-null int64
Date                                           1504150 non-null object
Day_of_Week                                    1504150 non-null int64
Time                                           1504033 non-null object
Road_Type                                      1504150 non-null object
Speed_limit                                    1504150 non-null int64
Pedestrian_Crossing-Physical_Facilities        1504116 non-null object
Light_Conditions                               1504150 non-null object
Weather_Conditions                    

### Renaming columns

In [51]:
column_names = {'Longitude': 'lon', 'Latitude': 'lat', 'Accident_Severity': 'severity', 
                'Number_of_Vehicles': 'num_vehicles', 'Number_of_Casualties': 'num_casualties',
                'Date': 'date', 'Day_of_Week': 'doy', 'Time': 'time', 'Road_Type': 'road_type',
                'Pedestrian_Crossing-Physical_Facilities': 'ped_crossing', 'Light_Conditions': 'light_cond',
                'Weather_Conditions': 'weather', 'Road_Surface_Conditions': 'road_cond',
                'Carriageway_Hazards': 'hazards', 'Urban_or_Rural_Area': 'urb_or_rur',
                'Did_Police_Officer_Attend_Scene_of_Accident': 'police_presence', 'Year': 'year'}

In [52]:
df.rename(columns=column_names, inplace=True)

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1504150 entries, 0 to 1504149
Data columns (total 18 columns):
lon                1504049 non-null float64
lat                1504049 non-null float64
severity           1504150 non-null int64
num_vehicles       1504150 non-null int64
num_casualties     1504150 non-null int64
date               1504150 non-null object
doy                1504150 non-null int64
time               1504033 non-null object
road_type          1504150 non-null object
Speed_limit        1504150 non-null int64
ped_crossing       1504116 non-null object
light_cond         1504150 non-null object
weather            1504024 non-null object
road_cond          1502192 non-null object
hazards            1504121 non-null object
urb_or_rur         1504150 non-null int64
police_presence    1501228 non-null object
year               1504150 non-null int64
dtypes: float64(2), int64(7), object(9)
memory usage: 218.0+ MB


### Given the very low number of Nans we will be dropping all the rows containing them to make the analysis and prediction easier

In [54]:
len(df.dropna())/len(df)

0.9969138716218462

In [55]:
df.dropna(inplace=True)

### Converting date column to datetime

In [56]:
df.date.head()

0    19/01/2012
1    04/01/2012
2    10/01/2012
3    18/01/2012
4    17/01/2012
Name: date, dtype: object

In [57]:
pd.to_datetime(df.date)

0         2012-01-19
1         2012-04-01
2         2012-10-01
3         2012-01-18
4         2012-01-17
             ...    
1504145   2007-12-20
1504146   2007-12-21
1504147   2007-12-23
1504148   2007-12-23
1504149   2007-12-22
Name: date, Length: 1499508, dtype: datetime64[ns]

### We got no errors therefore the dates are all formatted in the same way

In [58]:
df.date = pd.to_datetime(df.date)

### Removing minutes and keeping only the hours from Time column. Going to the next hour if minutes are > 30

In [59]:
func = lambda x: int(x[:-3]) if int(x[-2:]) <= 30 else int(x[:-3]) + 1
df.time = df.time.apply(func)

### Checking .head again

In [60]:
df.head()

,lon,lat,severity,num_vehicles,num_casualties,date,doy,time,road_type,Speed_limit,ped_crossing,light_cond,weather,road_cond,hazards,urb_or_rur,police_presence,year
0,-0.169101,51.493429,3,2,1,2012-01-19,5,21,Single carriageway,30,Pedestrian phase at traffic signal junction,Darkness: Street lights present and lit,Fine without high winds,Dry,None,1,Yes,2012
1,-0.200838,51.517931,3,2,1,2012-04-01,4,17,Single carriageway,30,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Dry,None,1,Yes,2012
2,-0.188636,51.487618,3,2,1,2012-10-01,3,10,One way street,30,non-junction pedestrian crossing,Daylight: Street light present,Fine without high winds,Dry,None,1,Yes,2012
3,-0.200259,51.514325,3,1,1,2012-01-18,4,12,Single carriageway,30,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,None,1,Yes,2012
4,-0.183773,51.497614,3,1,1,2012-01-17,3,20,Single carriageway,30,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Dry,None,1,Yes,2012


### Urban or rural

In [61]:
df.urb_or_rur.value_counts()

1    968908
2    530565
3        35
Name: urb_or_rur, dtype: int64

### Since 3 corresponds to undefined and it's only present in 35 observation we will be dropping the relevant rows.

In [62]:
df = df[df.urb_or_rur != 3]

### Types are fine and columns were correctly renamed. We will save the df to a new .csv file for the analysis and prediction

In [63]:
df.to_csv('../data/clean_data.csv')